In [ ]:

    # convert the image to hue 
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # extract the green colour from the hue
    lower_mask_hue = img_hsv [:,:,0] >= 25
    upper_mask_hue = img_hsv [:,:,0] <= 75
    saturation_lower = img_hsv [:,:,1] >= 40
    saturation_upper = img_hsv[:,:,1] <= 255
    value_lower = img_hsv[:,:,2]>= 50
    value_upper  = img_hsv[:,:,2]<=255
    mask = lower_mask_hue*upper_mask_hue*saturation_lower*saturation_upper*value_lower*value_upper
    red = img_hsv[:,:,0]*mask
    green = img_hsv[:,:,1]*mask
    blue = img_hsv[:,:,2]*mask
    mask = np.dstack((red,green,blue))
    mask = cv2.normalize(mask, None, alpha=0,beta =200, norm_type=cv2.NORM_MINMAX)
    return mask




In [ ]:
plt.imshow(segment_image(dataset[564]))

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
import os
import cv2
from glob import glob
images = []
labels = []
for dirname, folders,_ in os.walk('/kaggle/input/plant-seedlings-classification/train'):
    for folder in folders:
        path = os.path.join(dirname,folder)
        for image_path in glob(os.path.join(path, "*.png")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = segment_image(image)
            image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
            images.append(image)
            labels.append(folder)


In [ ]:
#change the labels to a categorical label into a numeric label 
from keras.utils import np_utils
from sklearn import preprocessing
labels = pd.DataFrame(labels)
le = preprocessing.LabelEncoder()
le.fit(labels[0])
encodeTrainLabels = le.transform(labels[0])
classes = np_utils.to_categorical(encodeTrainLabels)
images = np.asarray(images)
classes = np.asarray(classes).astype('int').reshape((4750,12))

In [ ]:
# Flatten the output layer to 1 dimension
from keras import applications
import tensorflow as tf
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
model = tf.keras.applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in model.layers[-5:]:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation="softmax")(x) 

model_final = Model(model.input, predictions)

model_final.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()


In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(images, classes, 
                                                test_size=0.2, random_state=2022, 
                                                stratify = classes)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically 
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )  
datagen.fit(trainX)

In [ ]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("vgg16_1", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
def callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'vgg16',
        monitor = 'val_loss', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', min_delta = 1e-2, patience = 8, verbose = 1,
        mode = 'min', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

In [ ]:

vgghist = model_final.fit(datagen.flow(trainX,trainY),validation_data=(testX,testY), steps_per_epoch = 110, epochs = 10,callbacks = callbacks())